In [1]:
import heapq
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import os

from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

from glob import glob

In [2]:
base_dir = '../data/dane_slowa_klucze/'

In [3]:
glob(base_dir + '*')

['../data/dane_slowa_klucze/interia',
 '../data/dane_slowa_klucze/natemat',
 '../data/dane_slowa_klucze/onet',
 '../data/dane_slowa_klucze/wp',
 '../data/dane_slowa_klucze/fronda',
 '../data/dane_slowa_klucze/gw',
 '../data/dane_slowa_klucze/rp_arch',
 '../data/dane_slowa_klucze/rp']

In [4]:
summary = pd.read_csv(base_dir + 'interia/GATT/art.csv', sep='\t')

In [5]:
summary.head()

,id,data,url,słowa klucze,nazwa pliku,liczba komentarzy
0,1,2016-12-23,http://biznes.interia.pl/makroekonomia/news/sa...,interia_1.txt,GATT (1),118
1,2,2017-05-26,http://biznes.interia.pl/makroekonomia/news/wi...,interia_2.txt,"CETA (1), GATT (1), WTO (12)",11
2,3,2018-07-12,https://biznes.interia.pl/makroekonomia/news/d...,interia_3.txt,GATT (4),10
3,4,2014-06-30,https://biznes.interia.pl/wiadomosci/news/na-s...,interia_4.txt,"GATT (1), WTO (7)",32
4,5,2016-08-12,https://fakty.interia.pl/news-wto-potwierdza-z...,interia_5.txt,"GATT (1), WTO (5)",59


In [6]:
glob(base_dir + '/interia/CEFTA/art/*')

['../data/dane_slowa_klucze//interia/CEFTA/art/interia_8.txt',
 '../data/dane_slowa_klucze//interia/CEFTA/art/interia_5.txt',
 '../data/dane_slowa_klucze//interia/CEFTA/art/interia_4.txt',
 '../data/dane_slowa_klucze//interia/CEFTA/art/interia_6.txt',
 '../data/dane_slowa_klucze//interia/CEFTA/art/interia_7.txt',
 '../data/dane_slowa_klucze//interia/CEFTA/art/interia_3.txt',
 '../data/dane_slowa_klucze//interia/CEFTA/art/interia_2.txt',
 '../data/dane_slowa_klucze//interia/CEFTA/art/interia_1.txt']

In [7]:
article = pd.read_csv(base_dir + 'interia/CEFTA/art/interia_1.txt', sep='\n\r', header=None)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [8]:
article.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 1 columns):
0    80 non-null object
dtypes: object(1)
memory usage: 720.0+ bytes


In [9]:
article.head(), article.tail()

(                                                   0
 0                              Aktywność gospodarcza
 1  W I kwartale 2000 roku produkt krajowy brutto ...
 2  Zmniejsza się tempo wzrostu popytu krajowego z...
 3  W II półroczu 1999 roku popyt krajowy zwiększy...
 4  Zwolnienie tempa wzrostu spożycia wynika z niż...,
                                                     0
 75  - rozwój sieci prywatnego i publicznego pośred...
 76  - przyspieszenie prac nad projektem ustawy o k...
 77                             4. Zmiany prawa pracy:
 78  - ograniczenie możliwości łączenia pobierania ...
 79  Sprawdź:  zobacz tanie noclegi w Poznaniu na e...)

## Check if last sentence value
Seems like some might be just an advertisement

In [10]:
[
    print(pd.read_csv(path, sep='\t', header=None).values[-1], pd.read_csv(path, sep='\t', header=None).shape)
    for path in sorted(glob(base_dir + 'interia/GATT/art/*'))
]

['Sprawdź:  hale całoroczne najlepszy wybór, https://sii.pl/oddzialy/krakow/, Reserved promocje - Zrabatowani'] (29, 1)
['Sprawdź:  Hosting od Kei.pl'] (17, 1)
['Sprawdź:  hale całoroczne najlepszy wybór, https://sii.pl/oddzialy/krakow/, Reserved promocje - Zrabatowani'] (16, 1)
['Dlatego trudno wskazać jedną przyczynę obecnego polskiego eksportowego boomu. Można go tłumaczyć przymusem w sytuacji, w której w kraju na początku lat 90. oraz 10 lat później gwałtownie zaczął słabnąć popyt wewnętrzny. Doszły do tego korzystne wówczas zmiany w przeliczeniach walutowych (deprecjacja złotego), trwające do połowy 2004 r., oraz poprawa koniunktury światowej. Do rozkwitu przyczyniło się także wstąpienie Polski do Unii Europejskiej. Uprościło ono prowadzenie działalności gospodarczej przez rodzime firmy na wspólnym rynku, zniosło ograniczenia i limity, tak ważne np. w eksporcie towarów rolno-spożywczych.'] (13, 1)
['Sprawdź:  hale całoroczne najlepszy wybór, https://sii.pl/oddzialy/krakow/, Reserv

[None, None, None, None, None, None, None, None, None, None, None, None]

## Read articles from data

In [11]:
class ArticlesOrganizer:
    def __init__(self, base_dir):
        self.base_dir = base_dir
        self.articles_header = ['id', 'site', 'topic', 'file_name']
        self.list_of_sites = self.get_list_of_sites()
        self.list_of_topics = self.get_list_of_topics()
#         self.articles = self.get_articles()
        
    def get_list_of_sites(self):
        return [
            site 
            for site in sorted(os.listdir(self.base_dir)) if site != '.DS_Store'
        ]
    
    def get_list_of_topics(self):
        return sorted(os.listdir(self.base_dir + self.list_of_sites[0]))
    
    def get_articles(self):
        articles = [
            [site, topic, base_dir + site + '/' + topic + '/art/' + article] 
            for site in self.list_of_sites 
            for topic in self.list_of_topics
            for article in os.listdir(self.base_dir + site + '/' + topic + '/art/')
        ]
        
        for counter, article in enumerate(articles):
            article.insert(0, counter + 1) 
        articles.insert(0, self.articles_header)
        
        return np.asarray(articles)

In [12]:
organizer = ArticlesOrganizer(base_dir)

In [13]:
articles = organizer.get_articles()

In [14]:
articles

array([['id', 'site', 'topic', 'file_name'],
       ['1', 'fronda', 'CETA',
        '../data/dane_slowa_klucze/fronda/CETA/art/fronda_40.txt'],
       ['2', 'fronda', 'CETA',
        '../data/dane_slowa_klucze/fronda/CETA/art/fronda_41.txt'],
       ...,
       ['4135', 'wp', 'WTO',
        '../data/dane_slowa_klucze/wp/WTO/art/wp_122.html'],
       ['4136', 'wp', 'WTO',
        '../data/dane_slowa_klucze/wp/WTO/art/wp_14.html'],
       ['4137', 'wp', 'WTO',
        '../data/dane_slowa_klucze/wp/WTO/art/wp_43.html']], dtype='<U58')

In [15]:
data = pd.DataFrame(articles[1:], columns = articles[0])

In [16]:
data.head()

,id,site,topic,file_name
0,1,fronda,CETA,../data/dane_slowa_klucze/fronda/CETA/art/fron...
1,2,fronda,CETA,../data/dane_slowa_klucze/fronda/CETA/art/fron...
2,3,fronda,CETA,../data/dane_slowa_klucze/fronda/CETA/art/fron...
3,4,fronda,CETA,../data/dane_slowa_klucze/fronda/CETA/art/fron...
4,5,fronda,CETA,../data/dane_slowa_klucze/fronda/CETA/art/fron...


## Check articles from interia
Hint: They're already txt files

In [17]:
interia = data[(data.site == 'interia')]
interia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 663 to 1182
Data columns (total 4 columns):
id           520 non-null object
site         520 non-null object
topic        520 non-null object
file_name    520 non-null object
dtypes: object(4)
memory usage: 20.3+ KB


In [18]:
df_interia = []
for filename in interia.file_name:
    try:
        df_interia.append(pd.read_csv(filename, sep='\n\r', header=None))
    except ParserError:
        continue

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [19]:
df_interia[0]

,0
0,Wicepremier i minister gospodarki Janusz Stein...
1,"""Chciałbym, jako minister gospodarki, aby zain..."
2,REKLAMA
3,"""Poziom sprzedaży na rynkach zewnętrznych jest..."
4,Strata netto Daewoo-FSO Motor SA wzrosła w 200...
5,Od początku roku do końca sierpnia Daewoo sprz...
6,Daewoo-FSO Motor SA ma zapas części do produkc...
7,We wtorek wstrzymanie dostaw części zmusiło sz...
8,Tymczasem prawdopodobnie już jutro wierzyciele...


## Compare two polish stopwords sets
* `many_stop_words.get_stop_words`
* `nltk.corpus.stopwords.words('polish')`

    For nltk stopwords, you need to put this file:
    [Polish stopwords](https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt)
    
    Into your nltk library at:
    `nltk_data/corpora/stopwords/`

In [20]:
polish_stopwords = get_stop_words('pl')

In [21]:
polish_stopwords_nltk = list(stopwords.words('polish'))

In [22]:
len(polish_stopwords), len(polish_stopwords_nltk)

(274, 350)

# Stemming using pystempel

https://github.com/dzieciou/pystempel

In [23]:
from stempel import StempelStemmer

In [24]:
stemmer = StempelStemmer.polimorf()

Loading: 100%|█████████▉| 11366702/11368252 [00:10<00:00, 1096879.94bytes/s]

In [25]:
for word in df_interia[0].values[0][0].split():
    if len(word) > 2:
        print(word, '\t\t\t', stemmer.stem(word))

Wicepremier 			 Wicepremier
minister 			 minister
gospodarki 			 gospodarka
Janusz 			 Jazić
Steinhoff 			 Steinhoff
powiedział, 			 powiedział,
byłby 			 być
zadowolony, 			 zadowolony,
gdyby 			 gdybyć
zainteresowanie 			 zainteresowanie
fabryką 			 fabryka
Daewoo-FSO 			 Daewoo-FSO
Motor 			 Motor
Warszawie 			 Warszawa
wyraził 			 wyrazić
jeden 			 jeden
producentów 			 producent
koreańskich. 			 koreańskich.
Tymczasem 			 Tymczas
prawdopodobnie 			 prawdopodobnia
już 			 ć
jutro 			 jutro
wierzyciele 			 wierzyciel
Daewoo 			 Daewosić
Motor 			 Motor
Co. 			 Co.
przedstawią 			 przedstawić
rezultaty 			 rezultat
rozmów 			 rozmówić
General 			 General
Motors 			 Motors
(GM) 			 (GM)
sprawie 			 sprawa
sprzedaży 			 sprzedaż
południowokoreańskiego 			 południowokoreański
koncernu 			 koncern
Amerykanom. 			 Amerykanom.
Polsce 			 Polsce
jest 			 być
zapas 			 zapas
części 			 część
produkcji 			 produkcja
oraz 			 oraz
części 			 część
zamiennych 			 zamienny
serwisu, 			 serwisu,


Loading: 100%|██████████| 11368252/11368252 [00:30<00:00, 1096879.94bytes/s]

In [27]:
def stem_sentence(sentence) -> str:
    stemmed_sentence = [
        stemmer.stem(word)
        for word in word_tokenize(sentence.lower()) if (word is not None) and (len(word) > 2)
    ]
    filtered_sentece = filter(None, stemmed_sentence)
    return " ".join(filtered_sentece)

In [28]:
def calculate_sentence_weight(sentences, frequency_table) -> dict:
    sentence_weight = dict()
    sent_index = 10
    
    for sentence in sentences:
        sentence_wordcount = len(word_tokenize(sentence))
        sentence_wordcount_without_stop_words = 0
        stemmed_sentence = stem_sentence(sentence)
        
        for word_weight in frequency_table:
            if word_weight in stemmed_sentence.lower():
                sentence_wordcount_without_stop_words += 1
                
                if sentence[:sent_index] in sentence_weight:
                    sentence_weight[sentence[:sent_index]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:sent_index]] = frequency_table[word_weight]
                    
        sentence_weight[sentence[:sent_index]] = sentence_weight[sentence[:sent_index]] \
            / sentence_wordcount_without_stop_words

    return sentence_weight

In [29]:
def create_dictionary_table(text) -> dict:
    stop_words = polish_stopwords
    words = word_tokenize(text)
    word_length_threshold = 3
    
    frequency_table = {}
    for word in words:
        if len(word) >= word_length_threshold:
            word = stemmer.stem(word)
        if word in stop_words or word is None:
            continue
        word = word.lower()
        if word in frequency_table:
            frequency_table[word] += 1
        else:
            frequency_table[word] = 1
            
    return frequency_table

In [30]:
def get_article_summary(sentences, sentence_weight, percentage=.25):
    no_of_sentences_in_summary = int(len(sentences) * percentage)
    sentence_counter = 0
    article_summary = ''
    sent_index = 10
    sentences_in_summary = heapq.nlargest(no_of_sentences_in_summary, sentence_weight, key=sentence_weight.)
    for sentence in sentences:
        if sentence[:sent_index] in sentences_in_summary:
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

In [52]:
def filter_short_sentences(sentences, threshold=30):
    return [sentence for sentence in sentences if len(sentence) > threshold]

In [46]:
def exctract_sentences_from_article(article):
    sentences = []
    for i, article in enumerate(article):
        sentences += sent_tokenize(article)
    return sentences

In [47]:
def get_articles_from_df(dataframe):
    return [
        dataframe[i].values.squeeze()
        for i in range(len(dataframe))
    ]

## Manual TFIDF

In [33]:
def _create_freq_matrix(sentences):
    stop_words = polish_stopwords
    word_length_threshold = 3
    sent_index = 10
    frequency_matrix = {}
    
    for sent in sentences:
        words = word_tokenize(sent)
        freq_table = {}
        for word in words:
            if len(word) >= word_length_threshold:
                word = stemmer.stem(word)
            if word in stop_words or word is None:
                continue
            word = word.lower()
            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1
        frequency_matrix[sent[:sent_index]] = freq_table
    return frequency_matrix

In [27]:
def _create_tf_matrix(frequency_matrix):
    tf_matrix = {}
    for sent, f_table in frequency_matrix.items():
        tf_table = {}
        
        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence
        tf_matrix[sent] = tf_table
    return tf_matrix

In [28]:
def _create_documents_per_words(frequency_matrix):
    word_per_doc_table = {}
    for sent, f_table in frequency_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1
    return word_per_doc_table

In [29]:
def _create_idf_matrix(
    frequency_matrix, 
    count_doc_per_words, 
    total_documents
):
    idf_matrix = {}
    for sent, f_table in frequency_matrix.items():
        idf_table = {}
        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))
        idf_matrix[sent] = idf_table
    return idf_matrix

In [30]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}
    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}
        
        for (word1, value1), (word2, value2) in zip(f_table1.items(), f_table2.items()):
            tf_idf_table[word1] = float(value1 * value2)
            
        tf_idf_matrix[sent1] = tf_idf_table
        
    return tf_idf_matrix

In [31]:
def _score_sentences(tf_idf_matrix):
    values = {}
    for sent, f_table in tf_idf_matrix.items():
        sentence_score = 0
        count_words = len(f_table)
        
        for word, score in f_table.items():
            sentence_score += score
        
        values[sent] = sentence_score / count_words
        
    return values

In [83]:
int(np.ceil(6.7))

7

In [91]:
def _get_top_sentences(sentences, scores, percentage=.25):
    no_of_sentences_in_summary = np.ceil(len(sentences) * percentage).astype(int)
    sentence_counter = 0
    article_summary = ''
    sent_index = 10
    sentences_in_summary = heapq.nlargest(no_of_sentences_in_summary, scores, key=scores.get)
    for sentence in sentences:
        if sentence[:sent_index] in sentences_in_summary:
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

In [98]:
def pipeline_article_summary(article, percentage=0.25, return_original=False):
    sentences = exctract_sentences_from_article(article=article)
    sentences = filter_short_sentences(sentences=sentences)
    
    article_original = " ".join(sentences)
    
    frequency_matrix = _create_freq_matrix(sentences)
    tf_matrix = _create_tf_matrix(frequency_matrix)
    
    count_doc_per_words = _create_documents_per_words(frequency_matrix)
    idf_matrix = _create_idf_matrix(
        frequency_matrix,
        count_doc_per_words,
        len(sentences)
    )
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    
    scores = _score_sentences(tf_idf_matrix)
    article_summary = _get_top_sentences(sentences, scores, percentage)
    if return_original:
        return article_summary, article_original
    return article_summary

## Example summarization

In [99]:
articles = get_articles_from_df(df_interia)

In [100]:
article_summary, article_original = pipeline_article_summary(articles[0], 0.3, True)
article_summary

' "Chciałbym, jako minister gospodarki, aby zainteresowanie tą fabryką wyraził jeden z producentów koreańskich. Zwróciłem się nawet ze stosownym listem i zaproszeniem do rozmów" - powiedział minister gospodarki Janusz Steinhoff. Steinhoff powiedział, że sytuację fabryki w Warszawie pogarszają także decyzje macierzystej firmy w Korei. "Poziom sprzedaży na rynkach zewnętrznych jest ograniczany w sposób niezrozumiały. Strata netto Daewoo-FSO Motor SA wzrosła w 2000 roku do 2,03 mld zł z 28 mln zł w 1999 roku. "Mamy zapas części do produkcji i części zamiennych do serwisu. Na pewno wystarczą na kilka najbliższych miesięcy"'

In [101]:
print('Summary length', len(article_summary))
print('Original length', len(article_original))
print('Percentage', len(article_summary) / len(article_original))

Summary length 625
Original length 3227
Percentage 0.1936783390145646


# Pipeline

#### Original

In [102]:
" ".join(articles[2])

'O 8 groszy więcej płacimy od dziś za litr benzyny i oleju napędowego z Rafinerii Gdańskiej. To kolejny producent podnoszący ceny - od wczoraj droższe są produkty Polskiego Koncernu Naftowego ORLEN. Benzyny i olej napędowy z Płocka podrożały również o 8 groszy. Szansą za zahamowanie podwyżek cen paliw będzie zniesienie od jutra cen na importowane paliwa. Na sprowadzane z krajów Unii Europejskiej, EFTA, CEFTA, Litwy, Estonii, Wysp Owczych i Turcji będzie obowiązywać zerowa stawka. O obniżce rząd zdecydował w ubiegłym tygodniu. Krok ten ma pomóc w walce z inflacją. REKLAMA Teraz - w przypadku benzyny - stawka celna wynosi 3 proc., a oleju napędowego 4 proc. Decyzja rządu zaledwie o kilkadziesiąt dni przyspiesza redukcję stawek, która i tak - zgodnie z umową z Unią Europejską - zaczęłaby obowiązywać 1 stycznia 2001 roku. Zdaniem ekspertów, polscy kierowcy niewiele zyskają na obniżce ceł, bo może to doprowadzić jedynie do spadku cen paliw na stacjach o około 4 grosze na litrze.'

#### Summarized

In [103]:
pipeline_article_summary(article=articles[2])

' Szansą za zahamowanie podwyżek cen paliw będzie zniesienie od jutra cen na importowane paliwa. O obniżce rząd zdecydował w ubiegłym tygodniu. Krok ten ma pomóc w walce z inflacją.'

## Summarize all interia articles

In [106]:
for i, article in enumerate(articles):
    print('Summary for text', i)
    print('=' * 97)
    summary = pipeline_article_summary(article=article, percentage=0.2)
    print(summary)
    print('=' * 30, 'Percentage of original text :', np.round(len(summary) / len(" ".join(article)) * 100), '%', '=' * 30)

Summary for text 0
 Steinhoff powiedział, że sytuację fabryki w Warszawie pogarszają także decyzje macierzystej firmy w Korei. "Poziom sprzedaży na rynkach zewnętrznych jest ograniczany w sposób niezrozumiały. Strata netto Daewoo-FSO Motor SA wzrosła w 2000 roku do 2,03 mld zł z 28 mln zł w 1999 roku. "Mamy zapas części do produkcji i części zamiennych do serwisu. Na pewno wystarczą na kilka najbliższych miesięcy"
============================== Percentage of original text : 12.0 % ==============================
Summary for text 1
 GMOs) - poinformował wiceminister rolnictwa Jerzy Plewa. Dzisiaj nie przewiduje się jeszcze ostatecznych decyzji. Dyskusja o reformie rynków tytoniu i cukru potrwa zapewne do 2004 roku. Więc można znaleźć tu wspólny interes - podkreślił.
============================== Percentage of original text : 8.0 % ==============================
Summary for text 2
 Szansą za zahamowanie podwyżek cen paliw będzie zniesienie od jutra cen na importowane paliwa. Krok ten ma 

 Władysław Kosiniak-Kamysz /Jakub Porzycki /Agencja FORUM Porozumienie ma być zawarte w październiku podczas szczytu UE-Kanada. Przeciwny CETA jest rząd Walonii, czyli francuskojęzycznego regionu Belgii. Zdaniem urzędników unijnych sprzeciw Belgów dyktowany jest względami politycznymi. Protesty przeciw umowie CETA odbyły się m.in. - zobacz debatę Interii i Uniwersytetu Jagiellońskiego (od 14:50):
============================== Percentage of original text : 10.0 % ==============================
Summary for text 9
 Chlorowane kurczaki trafią do polskich sklepów? Jej wprowadzenie to były lata walki. Sonda: Co wiemy o umowie CETA (Interia.tv) Interia.tv Zasada ostrożności w umowie CETA nie jest wystarczająco mocno osadzona. RZĄD WYSŁUCHA INFORMACJI O UMOWIE MIĘDZY UE A KANADĄ To jest dużo bardziej subtelne. Zasada ostrożności będzie sukcesywnie rozmontowywana. - Mogą to być tego typu sytuacje. To wyspecjalizowane grupy prawników. Nikt przypadkowy tam nie trafia? I to stało się udziałem Kan

 Premier Belgii Charles Michel /AFP "Cieszę się z dobrych informacji przekazanych przez premiera Charlesa Michela. sądu inwestycyjnego i importu produktów rolnych. Schinas nie chciał się wypowiadać w sprawie nowej daty szczytu UE-Kanada. Ten arbitraż uznawany jest przez przeciwników CETA za wzmocnienie wpływów wielkich korporacji. KE w lipcu zaproponowała państwom UE podpisanie umowy. Ostateczna ratyfikacja będzie należeć do parlamentów krajów unijnych i Kanady.
============================== Percentage of original text : 11.0 % ==============================
Summary for text 25
 Wiążący dokument interpretacyjny CETA zliberalizuje handel usługami między UE a Kanadą Ten arbitraż uznawany jest przez przeciwników CETA za wzmocnienie wpływów wielkich korporacji. Tego dotyczyły też najważniejsze zastrzeżenia Walonii. Belgijską deklarację zaakceptowały w czwartek pozostałe państwa UE.
============================== Percentage of original text : 10.0 % ==============================
Summary f

 JPK póki co z wymogiem bezpiecznego e-podpisu Kolejna tura przedsiębiorców zmierzy się z cyfryzacją fiskusa. Standardem staje się przesyłka przez internet danych o VAT. Jak przygotować się do nadchodzących zmian? Wszystko zależy od skali działalności i wielkości zatrudnienia w przedsiębiorstwie. Na start liczymy pracowników i obroty w firmie Co należy rozumieć przez pojęcie mały i średnie przedsiębiorstwo? Muszą zostać spełnione dwa kryteria równocześnie w co najmniej jednym z dwóch ostatnich lat obrotowych. Głównym wskaźnikiem, który zawsze jest brany pod uwagę zawsze to liczba pracowników. Czy to dobrze, że rząd porzucił jednolity podatek? Kompresja, podpis i strzał na bramkę Ustawodawca wskazał również jego rozmiar. Oferta obejmuje rozwiązania zarówno w wersji stacjonarnej, jak i w wersji chmurowej. w miesiącach styczeń-listopad 2016. Przewidywane wzrosty cen paliw odbiją się na zyskowności branży transportowej. Ma to istotne znaczenie dla potencjału handlowego. Można założyć, że t

 Bruksela: Komisja handlu PE poparła umowę CETA /123RF/PICSEL To wydaje się być jasno określone w deklaracji" - zaznaczył Martin. Sonda: Co wiemy o umowie CETA (Interia.tv) Interia.tv Jak zaznaczył, jeśli jakiś kraj unijny zdecydował, że chce, by operatorem np. "To absolutnie pewne" - podkreślił. Negocjacje w tej sprawie zakończyły się we wrześniu 2014 roku. Interia i UJ zapraszają na debatę /INTERIA.PL
============================== Percentage of original text : 9.0 % ==============================
Summary for text 66
 dr Adam Kirpsza, Uniwersytet Jagielloński Rafał Górski, prezes Instytutu Spraw Obywatelskich
============================== Percentage of original text : 13.0 % ==============================
Summary for text 67
 "To jest naprawdę bardzo, bardzo mocne zabezpieczenie. Kryzys zdolności decyzyjnej w UE Podpisanie umowy UE-Kanada blokował dotąd belgijski region Walonia. Walonia zgłaszała zastrzeżenia m.in. Ostateczna ratyfikacja będzie należeć do parlamentów krajów unijnych

...

 - Same negocjacje wymagają od państw aspirujących spełnienia wielu kryteriów. Rosji - jest wcześniejsze przystąpienie do OECD. - To było w interesie Rosji, jak i pozostałych państw. - Bardzo bym chciał, żeby to było możliwie szybko. - Tym bardziej, że z Moskwy są formułowane pewne obawy.
============================== Percentage of original text : 8.0 % ==============================
Summary for text 453
 Polska ma za wysokie ceny energii i gazu! Jej zdaniem UE w ten sposób złamała zasady WTO. Światowa Organizacja Handlu nie podzieliła jednak tego stanowiska. zasada TPA - third party access). udzielanie części przepustowości gazociągu konkurentom. Ile płaci się za prąd i gaz w Unii Europejskiej? W Polsce było to ok. 15 eurocentów. Trasa ta omija Bornholm od północy. Odpowiednie zgody na budowę wydały także Finlandia i Niemcy. Gazociąg o zdolności przesyłowej 55 mld m sześc.
============================== Percentage of original text : 7.0 % ==============================
Summary for te

 Antyamerykańska "koalicja stalowa" Podwyżka wejdzie w życie za dwa tygodnie.
============================== Percentage of original text : 7.0 % ==============================
Summary for text 468
 Rosyjski minister wyraził też przekonanie co do dalszego rozwoju dialogu energetycznego między Rosją a UE. Negocjacje akcesyjne rozpoczęła 14 lat temu.
============================== Percentage of original text : 12.0 % ==============================
Summary for text 469
 Polska ma za wysokie ceny energii i gazu! Jej zdaniem UE w ten sposób złamała zasady WTO. Światowa Organizacja Handlu nie podzieliła jednak tego stanowiska. zasada TPA - third party access). udzielanie części przepustowości gazociągu konkurentom. Ile płaci się za prąd i gaz w Unii Europejskiej? W Polsce było to ok. 15 eurocentów. Trasa ta omija Bornholm od północy. Odpowiednie zgody na budowę wydały także Finlandia i Niemcy. Gazociąg o zdolności przesyłowej 55 mld m sześc.
============================== Percentage of origin

 Za nowa ustawą głosowało 348 kongresmenów, przeciw było 79. Otwiera to drogę do obłożenia dodatkowym cłem chińskich towarów wjeżdżających do USA. Wiadomość ta sprawiła, że z frustracji zapragnąłem tłuc głową o ścianę. Uderzenie w juana będzie "katastrofą" Podobne kroki podejmują też inne kraje. Brazylijski minister finansów przestrzegł w tym tygodniu przed groźbą globalnej wojny walutowej. Brazylijski minister finansów przestrzegł w tym tygodniu przed groźbą globalnej wojny walutowej.
============================== Percentage of original text : 12.0 % ==============================
Summary for text 484
 Zgodnie z zapowiedziami o północy z czwartku na piątek na wschodzie USA (godz. Taryfy mają być dla Pekinu karą za wymuszanie transferu technologii od zagranicznych firm. Są wśród nich produkty rolno-spożywcze i samochody. Nie podał szczegółów dotyczących liczby i wartości towarów objętych cłami odwetowymi. Sprawdź:  hale całoroczne najlepszy wybór, https://sii.pl/oddzialy/krakow/, Rese

 Kryzys polityczny na Ukrainie przesłoni wtorkowy szczyt UE-Rosja w Brukseli. Berkut na Majdanie, pod przyniesioną przez protestujących flagą UE /PAP Pod presją Moskwy władze Ukrainy wstrzymały w listopadzie ub. Rosja odrzuca oskarżenia o wywieranie nacisków na swoich sąsiadów. Byliśmy świadkami prób protekcjonizmu i działań niezgodnych z zasadami WTO. Komisja Europejska prowadzi postępowanie antymonopolowe przeciwko Gazpromowi i przygotowuje zarzuty. UE jest najważniejszym odbiorcą rosyjskich surowców energetycznych. zużywanej w UE ropy i gazu pochodzi z Rosji.
============================== Percentage of original text : 11.0 % ==============================
Summary for text 503
 - Chodzi o podpisanie długookresowych umów na dostawy gazu ziemnego dla Ukrainy oraz jego tranzyt przez Ukrainę. An-124 to jeden z największych samolotów transportowych świata. Najpierw był produkowany w ZSRR, a potem w Rosji i na Ukrainie. - Chodzi o Polskę, Węgry i Słowację - powiedziała Tymoszenko.
=======

 Francja: Minister finansów ostrzega przed zimną wojną handlową Piątek, 16 listopada 2018 (11:09)
============================== Percentage of original text : 7.0 % ==============================
